In [ ]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.initializers import Constant, HeNormal
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Colab Notebooks/TFM/datosModelo.csv'

In [ ]:
df = pd.read_csv(path,sep=";")
df

## Análisis de las variables a predecir

In [ ]:
conteo_local = df.groupby('resultado_local').size()
conteo_local

In [ ]:
conteo_visitante = df.groupby('resultado_visitante').size()
conteo_visitante

In [ ]:
conteo_partido = df.groupby('resultado_partido').size()
conteo_partido

## Preprocesamiento

In [ ]:
# Eliminar la columna id_partido
df.drop(['jornada','id_indicadores_equipo_prepartido'], axis=1, inplace=True)

In [ ]:
# Separar las características (porcentajes) y la clase a predecir (resultado_partido)
X = df.drop(['resultado_partido','resultado_local','resultado_visitante'], axis=1)

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
y_resultado=df['resultado_partido']
y_resultado_local=df['resultado_local']
y_resultado_visitante=df['resultado_visitante']

## Predicciones para el ganador del partido

In [ ]:
opciones_epocas=[20,50,80]
opciones_batch=[16,48,80]
opciones_optimizador=["SGD","adam","RMSProp"]
opciones_callbacks=[0,1]
opciones_red=[1,2,3,4,5,6]


In [ ]:
# Codificar las etiquetas (resultado_partido)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_resultado)

# Convertir las etiquetas a one-hot encoding
y_categorical = to_categorical(y_encoded)

In [ ]:
resultsDF=pd.DataFrame(columns=["epocas","batch","optimizador","callbacks","red","Exactitud"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

In [ ]:


for i in range(len(opciones_epocas)):
  for j in range(len(opciones_batch)):
    for k in range(len(opciones_optimizador)):
      for l in range(len(opciones_callbacks)):
        for m in range(len(opciones_red)):

          if opciones_red[m]==1:
              model = Sequential([
                  Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
                  BatchNormalization(),
                  Dropout(0.3),
                  Dense(64, activation='relu'),
                  BatchNormalization(),
                  Dropout(0.3),
                  Dense(3, activation='softmax')
              ])
          elif opciones_red[m]==2:
              model = Sequential([
                Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.4),
                Dense(3, activation='softmax')
            ])
          elif opciones_red[m]==3:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64,
                      activation='relu',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(3,
                      activation='softmax',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01))
            ])
          elif opciones_red[m]==4:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(3,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01))
              ])
          elif  opciones_red[m]==5:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(3,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01))
            ])
          else:
            model = Sequential([
                Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
                BatchNormalization(),
                Dropout(0.5),
                Dense(256, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(32, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(3, activation='softmax')
            ])




          model.compile(optimizer=opciones_optimizador[k], loss="categorical_crossentropy", metrics=['accuracy'])

          # Callbacks
          early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
          reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5)

          if opciones_callbacks[l]==1:
            history = model.fit(X_train, y_train,
                                epochs=opciones_epocas[i], batch_size=opciones_batch[j], callbacks=[early_stopping, reduce_lr])
          else:
            history = model.fit(X_train, y_train,
                                epochs=opciones_epocas[i], batch_size=opciones_batch[j])

          loss, accuracy = model.evaluate(X_test, y_test)
          print(f"Exactitud en el conjunto de prueba: {accuracy:.4f}")
          resultsDF.loc[len(resultsDF)]=[opciones_epocas[i], opciones_batch[j], opciones_optimizador[k], opciones_callbacks[l], opciones_red[m], accuracy]

In [ ]:
resultsDF.to_csv('/content/drive/MyDrive/Colab Notebooks/TFM/modelos_resultado_partido.csv')

In [ ]:
# Modelo final
model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(3,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01))
            ])


model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])



history = model.fit(X_train, y_train,
                      epochs=20, batch_size=48)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
print(f"Exactitud en el conjunto de prueba: {accuracy:.4f}")

In [ ]:
# Acceder a los pesos de la primera capa
first_layer_weights = model.layers[0].get_weights()

# first_layer_weights es una lista que contiene los pesos y los sesgos (bias) de la capa
weights, biases = first_layer_weights

# Sumar los pesos absolutos de cada característica
feature_importance = np.sum(np.abs(weights), axis=1)

# Normalizar las importancias para facilitar la interpretación
feature_importance_normalized = feature_importance / np.sum(feature_importance)

# Ordenar las importancias de mayor a menor
sorted_indices = np.argsort(feature_importance_normalized)[::-1]

# Seleccionar los índices de las 10 variables más importantes
top_n = 30
top_indices = sorted_indices[:top_n]

# Obtener los nombres de las columnas correspondientes
top_features_labels = [df.columns[i] for i in top_indices]

# Imprimir las 10 variables más importantes con sus valores de importancia
print("Las variables con más importancia son:")
for idx, importance in zip(top_indices, feature_importance_normalized[top_indices]):
    print(f' {df.columns[idx]}; {importance:.4f}')





In [ ]:
# Predecir las probabilidades de las clases en el conjunto de prueba
y_pred_proba = model.predict(X_test)

# Decodificar las etiquetas predichas y reales
y_test_decoded = label_encoder.inverse_transform(np.argmax(y_test, axis=1))
y_pred_decoded = label_encoder.inverse_transform(np.argmax(y_pred_proba, axis=1))

# Mostrar las probabilidades para cada clase junto con la clase real
for i, (probs, true_class, pred_class) in enumerate(zip(y_pred_proba, y_test_decoded, y_pred_decoded)):
    print(f"Partido {i+1}:")
    print(f"  Clase real: {true_class}")
    print(f"  Clase predicha: {pred_class}")
    for class_name, prob in zip(label_encoder.classes_, probs):
        print(f"  Probabilidad de {class_name}: {prob:.4f}")

In [ ]:

valores = np.array([37.5,27.27,25.0,24.24,37.5,48.48,0.0,9.09,25.0,27.27,75.0,63.64,1.38,1.06,0.25,0.55,87.5,75.76,87.5,78.79,56.25,51.52,81.25,63.64,31.25,33.33,56.25,42.42,25.0,24.24,31.25,24.24,3.0,2.79,1.44,1.12,1.56,1.67,3.69,3.24,0.75,0.82,2.94,2.42,87.5,72.73,37.5,30.3,12.5,6.06,81.25,72.73,31.25,39.39,25.0,33.33,68.75,66.67,6.25,12.12,0.0,3.03,87.5,81.82,81.25,63.64,62.5,45.45,1.75,2.18,1.69,1.76,0.0,0.0,0.0,0.0,5.0,4.94,4.31,4.06,44.69,43.55,45.75,46.91,12.44,11.3,7.38,9.24,5.38,4.3,3.62,4.03,5.81,5.76,5.75,5.55,0.19,0.3,0.25,0.12,0.81,0.73,0.62,0.58,0.44,0.36,0.31,0.36,0.62,0.42,0.19,0.12,69.82,68.12,67.3,71.16,0.44,0.36,0.25,0.21,0.31,0.42,0.12,0.15,0.12,0.18,0.5,0.48,0.12,0.18,0.44,0.21,0.31,0.24,0.06,0.09,0.12,0.18,0.12,0.06,0.19,0.18,0.25,0.12,1.06,1.24,0.81,0.58,1.75,1.79,1.62,1.52,2.0,1.73,1.62,1.85,1.12,1.3,1.38,1.21,0.81,0.82,1.12,1.15,1.31,1.36,0.81,0.76])
valores = valores.reshape((1, 168))
posiciones_visitante=[165,161,157,153,149,145,141,137,133,129,125,121,117,97,99,101,103,105,107,109,111,113,115,167,163,159,155,151,147,143,139,135,131,127,123,119]
for index in posiciones_visitante:
    valores[0,index] += 3
    print("-------------------",str(index))

    # Predecir las probabilidades 
    y_pred_proba = model.predict(valores)

    # Decodificar las etiquetas predichas y reales
    y_pred_decoded = label_encoder.inverse_transform(np.argmax(y_pred_proba, axis=1))

    # Mostrar las probabilidades 
    for i, (probs,  pred_class) in enumerate(zip(y_pred_proba,  y_pred_decoded)):
        print(f"Partido {i+1}:")
        print(f"  Clase predicha: {pred_class}")
        for class_name, prob in zip(label_encoder.classes_, probs):
            print(f"  Probabilidad de {class_name}: {prob:.4f}")

    valores[0,index] -= 3

## Predicciones para los goles del local

In [ ]:
# Codificar las etiquetas (resultado_local)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_resultado_local)

# Convertir las etiquetas a one-hot encoding
y_categorical = to_categorical(y_encoded)

In [ ]:
resultsDF=pd.DataFrame(columns=["epocas","batch","optimizador","callbacks","red","Exactitud"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

In [ ]:


for i in range(len(opciones_epocas)):
  for j in range(len(opciones_batch)):
    for k in range(len(opciones_optimizador)):
      for l in range(len(opciones_callbacks)):
        for m in range(len(opciones_red)):

          if opciones_red[m]==1:
              model = Sequential([
                  Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
                  BatchNormalization(),
                  Dropout(0.3),
                  Dense(64, activation='relu'),
                  BatchNormalization(),
                  Dropout(0.3),
                  Dense(10, activation='softmax')
              ])
          elif opciones_red[m]==2:
              model = Sequential([
                Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.4),
                Dense(10, activation='softmax')
            ])
          elif opciones_red[m]==3:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64,
                      activation='relu',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(10,
                      activation='softmax',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01))
            ])
          elif opciones_red[m]==4:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(10,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01))
              ])
          elif  opciones_red[m]==5:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(10,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01))
            ])
          else:
            model = Sequential([
                Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
                BatchNormalization(),
                Dropout(0.5),
                Dense(256, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(32, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(10, activation='softmax')
            ])




          model.compile(optimizer=opciones_optimizador[k], loss="categorical_crossentropy", metrics=['accuracy'])

          # Callbacks
          early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
          reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5)

          if opciones_callbacks[l]==1:
            history = model.fit(X_train, y_train,
                                epochs=opciones_epocas[i], batch_size=opciones_batch[j], callbacks=[early_stopping, reduce_lr])
          else:
            history = model.fit(X_train, y_train,
                                epochs=opciones_epocas[i], batch_size=opciones_batch[j])

          loss, accuracy = model.evaluate(X_test, y_test)
          print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
          print(f"Exactitud en el conjunto de prueba: {accuracy:.4f}")
          resultsDF.loc[len(resultsDF)]=[opciones_epocas[i], opciones_batch[j], opciones_optimizador[k], opciones_callbacks[l], opciones_red[m], accuracy]

In [ ]:
resultsDF.to_csv('/content/drive/MyDrive/Colab Notebooks/TFM/modelos_resultado_local.csv')

In [ ]:
# Modelo final
model = Sequential([
                Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
                BatchNormalization(),
                Dropout(0.5),
                Dense(256, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(32, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(10, activation='softmax')
            ])


model.compile(optimizer="sgd", loss="categorical_crossentropy", metrics=['accuracy'])



history = model.fit(X_train, y_train,
                      epochs=20, batch_size=80)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
print(f"Exactitud en el conjunto de prueba: {accuracy:.4f}")

In [ ]:
# Acceder a los pesos de la primera capa
first_layer_weights = model.layers[0].get_weights()

# first_layer_weights es una lista que contiene los pesos y los sesgos (bias) de la capa
weights, biases = first_layer_weights

# Sumar los pesos absolutos de cada característica
feature_importance = np.sum(np.abs(weights), axis=1)

# Normalizar las importancias para facilitar la interpretación
feature_importance_normalized = feature_importance / np.sum(feature_importance)

# Ordenar las importancias de mayor a menor
sorted_indices = np.argsort(feature_importance_normalized)[::-1]

# Seleccionar los índices de las 10 variables más importantes
top_n = 30
top_indices = sorted_indices[:top_n]

# Obtener los nombres de las columnas correspondientes
top_features_labels = [df.columns[i] for i in top_indices]

# Imprimir las 10 variables más importantes con sus valores de importancia
print("Las variables con más importancia son:")
for idx, importance in zip(top_indices, feature_importance_normalized[top_indices]):
    print(f' {df.columns[idx]}; {importance:.4f}')





In [ ]:
# Predecir las probabilidades de las clases en el conjunto de prueba
y_pred_proba = model.predict(X_test)

# Decodificar las etiquetas predichas y reales
y_test_decoded = label_encoder.inverse_transform(np.argmax(y_test, axis=1))
y_pred_decoded = label_encoder.inverse_transform(np.argmax(y_pred_proba, axis=1))

# Mostrar las probabilidades para cada clase junto con la clase real
for i, (probs, true_class, pred_class) in enumerate(zip(y_pred_proba, y_test_decoded, y_pred_decoded)):
    print(f"Partido {i+1}:")
    print(f"  Clase real: {true_class}")
    print(f"  Clase predicha: {pred_class}")
    for class_name, prob in zip(label_encoder.classes_, probs):
        print(f"  Probabilidad de {class_name}: {prob:.4f}")

In [ ]:
valores = np.array([37.5,27.27,25.0,24.24,37.5,48.48,0.0,9.09,25.0,27.27,75.0,63.64,1.38,1.06,0.25,0.55,87.5,75.76,87.5,78.79,56.25,51.52,81.25,63.64,31.25,33.33,56.25,42.42,25.0,24.24,31.25,24.24,3.0,2.79,1.44,1.12,1.56,1.67,3.69,3.24,0.75,0.82,2.94,2.42,87.5,72.73,37.5,30.3,12.5,6.06,81.25,72.73,31.25,39.39,25.0,33.33,68.75,66.67,6.25,12.12,0.0,3.03,87.5,81.82,81.25,63.64,62.5,45.45,1.75,2.18,1.69,1.76,0.0,0.0,0.0,0.0,5.0,4.94,4.31,4.06,44.69,43.55,45.75,46.91,12.44,11.3,7.38,9.24,5.38,4.3,3.62,4.03,5.81,5.76,5.75,5.55,0.19,0.3,0.25,0.12,0.81,0.73,0.62,0.58,0.44,0.36,0.31,0.36,0.62,0.42,0.19,0.12,69.82,68.12,67.3,71.16,0.44,0.36,0.25,0.21,0.31,0.42,0.12,0.15,0.12,0.18,0.5,0.48,0.12,0.18,0.44,0.21,0.31,0.24,0.06,0.09,0.12,0.18,0.12,0.06,0.19,0.18,0.25,0.12,1.06,1.24,0.81,0.58,1.75,1.79,1.62,1.52,2.0,1.73,1.62,1.85,1.12,1.3,1.38,1.21,0.81,0.82,1.12,1.15,1.31,1.36,0.81,0.76])
valores = valores.reshape((1, 168))
posiciones_visitante=[165,161,157,153,149,145,141,137,133,129,125,121,117]
for index in posiciones_visitante:
    valores[0,index] += 3
    print("-------------------",str(index))

    # Predecir las probabilidades 
    y_pred_proba = model.predict(valores)

    # Decodificar las etiquetas predichas y reales
    y_pred_decoded = label_encoder.inverse_transform(np.argmax(y_pred_proba, axis=1))

    # Mostrar las probabilidades 
    for i, (probs,  pred_class) in enumerate(zip(y_pred_proba,  y_pred_decoded)):
        print(f"Partido {i+1}:")
        print(f"  Clase predicha: {pred_class}")
        for class_name, prob in zip(label_encoder.classes_, probs):
            print(f"  Probabilidad de {class_name}: {prob:.4f}")

    valores[0,index] -= 3

## Predicciones para los goles del visitante

In [ ]:
# Codificar las etiquetas (resultado_local)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y_resultado_visitante)

# Convertir las etiquetas a one-hot encoding
y_categorical = to_categorical(y_encoded)

In [ ]:
resultsDF=pd.DataFrame(columns=["epocas","batch","optimizador","callbacks","red","Exactitud"])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_categorical, test_size=0.2, random_state=42)

In [ ]:


for i in range(len(opciones_epocas)):
  for j in range(len(opciones_batch)):
    for k in range(len(opciones_optimizador)):
      for l in range(len(opciones_callbacks)):
        for m in range(len(opciones_red)):

          if opciones_red[m]==1:
              model = Sequential([
                  Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
                  BatchNormalization(),
                  Dropout(0.3),
                  Dense(64, activation='relu'),
                  BatchNormalization(),
                  Dropout(0.3),
                  Dense(9, activation='softmax')
              ])
          elif opciones_red[m]==2:
              model = Sequential([
                Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.4),
                Dense(9, activation='softmax')
            ])
          elif opciones_red[m]==3:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64,
                      activation='relu',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(9,
                      activation='softmax',
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01))
            ])
          elif opciones_red[m]==4:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(9,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01))
              ])
          elif  opciones_red[m]==5:
              model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(9,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      bias_initializer=Constant(0.1),
                      kernel_regularizer=l2(0.01),
                      bias_regularizer=l2(0.01))
            ])
          else:
            model = Sequential([
                Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
                BatchNormalization(),
                Dropout(0.5),
                Dense(256, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(32, activation='relu'),
                BatchNormalization(),
                Dropout(0.5),
                Dense(9, activation='softmax')
            ])




          model.compile(optimizer=opciones_optimizador[k], loss="categorical_crossentropy", metrics=['accuracy'])

          # Callbacks
          early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
          reduce_lr = ReduceLROnPlateau(factor=0.2, patience=5)

          if opciones_callbacks[l]==1:
            history = model.fit(X_train, y_train,
                                epochs=opciones_epocas[i], batch_size=opciones_batch[j], callbacks=[early_stopping, reduce_lr])
          else:
            history = model.fit(X_train, y_train,
                                epochs=opciones_epocas[i], batch_size=opciones_batch[j])

          loss, accuracy = model.evaluate(X_test, y_test)
          print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
          print(f"Exactitud en el conjunto de prueba: {accuracy:.4f}")
          resultsDF.loc[len(resultsDF)]=[opciones_epocas[i], opciones_batch[j], opciones_optimizador[k], opciones_callbacks[l], opciones_red[m], accuracy]

In [ ]:
resultsDF.to_csv('/content/drive/MyDrive/Colab Notebooks/TFM/modelos_resultado_visitante.csv')

In [ ]:
# Modelo final
model = Sequential([
                Dense(256,
                      activation='relu',
                      input_shape=(X_train.shape[1],),
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(128,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(64,
                      activation='relu',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01)),
                BatchNormalization(),
                Dropout(0.4),
                Dense(9,
                      activation='softmax',
                      kernel_initializer=HeNormal(),
                      kernel_regularizer=l2(0.01))
              ])


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=['accuracy'])



history = model.fit(X_train, y_train,
                      epochs=20, batch_size=80)

loss, accuracy = model.evaluate(X_test, y_test)
print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
print(f"Exactitud en el conjunto de prueba: {accuracy:.4f}")

In [ ]:
# Acceder a los pesos de la primera capa
first_layer_weights = model.layers[0].get_weights()

# first_layer_weights es una lista que contiene los pesos y los sesgos (bias) de la capa
weights, biases = first_layer_weights

# Sumar los pesos absolutos de cada característica
feature_importance = np.sum(np.abs(weights), axis=1)

# Normalizar las importancias para facilitar la interpretación
feature_importance_normalized = feature_importance / np.sum(feature_importance)

# Ordenar las importancias de mayor a menor
sorted_indices = np.argsort(feature_importance_normalized)[::-1]

# Seleccionar los índices de las 10 variables más importantes
top_n = 30
top_indices = sorted_indices[:top_n]

# Obtener los nombres de las columnas correspondientes
top_features_labels = [df.columns[i] for i in top_indices]

# Imprimir las 10 variables más importantes con sus valores de importancia
print("Las variables con más importancia son:")
for idx, importance in zip(top_indices, feature_importance_normalized[top_indices]):
    print(f' {df.columns[idx]}; {importance:.4f}')



In [ ]:
# Predecir las probabilidades de las clases en el conjunto de prueba
y_pred_proba = model.predict(X_test)

# Decodificar las etiquetas predichas y reales
y_test_decoded = label_encoder.inverse_transform(np.argmax(y_test, axis=1))
y_pred_decoded = label_encoder.inverse_transform(np.argmax(y_pred_proba, axis=1))

# Mostrar las probabilidades para cada clase junto con la clase real
for i, (probs, true_class, pred_class) in enumerate(zip(y_pred_proba, y_test_decoded, y_pred_decoded)):
    print(f"Partido {i+1}:")
    print(f"  Clase real: {true_class}")
    print(f"  Clase predicha: {pred_class}")
    for class_name, prob in zip(label_encoder.classes_, probs):
        print(f"  Probabilidad de {class_name}: {prob:.4f}")

In [ ]:

valores = np.array([37.5,27.27,25.0,24.24,37.5,48.48,0.0,9.09,25.0,27.27,75.0,63.64,1.38,1.06,0.25,0.55,87.5,75.76,87.5,78.79,56.25,51.52,81.25,63.64,31.25,33.33,56.25,42.42,25.0,24.24,31.25,24.24,3.0,2.79,1.44,1.12,1.56,1.67,3.69,3.24,0.75,0.82,2.94,2.42,87.5,72.73,37.5,30.3,12.5,6.06,81.25,72.73,31.25,39.39,25.0,33.33,68.75,66.67,6.25,12.12,0.0,3.03,87.5,81.82,81.25,63.64,62.5,45.45,1.75,2.18,1.69,1.76,0.0,0.0,0.0,0.0,5.0,4.94,4.31,4.06,44.69,43.55,45.75,46.91,12.44,11.3,7.38,9.24,5.38,4.3,3.62,4.03,5.81,5.76,5.75,5.55,0.19,0.3,0.25,0.12,0.81,0.73,0.62,0.58,0.44,0.36,0.31,0.36,0.62,0.42,0.19,0.12,69.82,68.12,67.3,71.16,0.44,0.36,0.25,0.21,0.31,0.42,0.12,0.15,0.12,0.18,0.5,0.48,0.12,0.18,0.44,0.21,0.31,0.24,0.06,0.09,0.12,0.18,0.12,0.06,0.19,0.18,0.25,0.12,1.06,1.24,0.81,0.58,1.75,1.79,1.62,1.52,2.0,1.73,1.62,1.85,1.12,1.3,1.38,1.21,0.81,0.82,1.12,1.15,1.31,1.36,0.81,0.76])
valores = valores.reshape((1, 168))
posiciones_visitante=[167,163,159,155,151,147,143,139,135,131,127,123,119]
for index in posiciones_visitante:
    valores[0,index] += 3
    print("-------------------",str(index))

    # Predecir las probabilidades 
    y_pred_proba = model.predict(valores)

    # Decodificar las etiquetas predichas y reales
    y_pred_decoded = label_encoder.inverse_transform(np.argmax(y_pred_proba, axis=1))

    # Mostrar las probabilidades 
    for i, (probs,  pred_class) in enumerate(zip(y_pred_proba,  y_pred_decoded)):
        print(f"Partido {i+1}:")
        print(f"  Clase predicha: {pred_class}")
        for class_name, prob in zip(label_encoder.classes_, probs):
            print(f"  Probabilidad de {class_name}: {prob:.4f}")

    valores[0,index] -= 3